# Chapter 27: Real-Time Features

Modern web applications increasingly demand real-time capabilities—live notifications, collaborative editing, instant messaging, and real-time dashboards. Next.js applications can leverage WebSockets for bidirectional communication, Server-Sent Events for server-to-client streaming, and modern real-time databases for seamless synchronization. Implementing these features requires careful consideration of connection management, scalability, and the hybrid nature of the App Router.

By the end of this chapter, you'll master WebSocket integration in Next.js Route Handlers, implement Server-Sent Events for live streaming, establish real-time data synchronization patterns, build live update mechanisms for UI components, configure push notifications for engagement, create collaborative features with operational transforms, and follow real-time architecture best practices.

## 27.1 WebSockets Integration

WebSockets provide full-duplex communication channels ideal for chat applications, live notifications, and collaborative editing.

### Socket.io with Next.js

Integrate Socket.io for robust real-time communication:

```typescript
// app/api/socket/route.ts
import { Server as NetServer } from 'http';
import { NextApiRequest } from 'next';
import { Server as ServerIO } from 'socket.io';
import { NextResponse } from 'next/server';

export const dynamic = 'force-dynamic';

export async function GET(req: Request) {
  if ((global as any).io) {
    return NextResponse.json({ success: true, status: 'already-running' });
  }

  const res = await fetch('http://localhost:3000').then(r => r.text()).catch(() => null);
  
  // Initialize Socket.io on the global object to prevent multiple instances
  const io = new ServerIO({
    path: '/api/socket/io',
    addTrailingSlash: false,
    cors: {
      origin: '*',
      methods: ['GET', 'POST'],
    },
  });

  io.on('connection', (socket) => {
    console.log('Client connected:', socket.id);

    // Join room based on user ID or session
    socket.on('join-room', (roomId: string) => {
      socket.join(roomId);
      console.log(`Socket ${socket.id} joined room ${roomId}`);
    });

    // Handle chat messages
    socket.on('send-message', async (data: { roomId: string; message: string; userId: string }) => {
      // Persist message to database
      const savedMessage = await db.message.create({
        data: {
          content: data.message,
          roomId: data.roomId,
          userId: data.userId,
        },
        include: { user: true },
      });

      // Broadcast to room
      io.to(data.roomId).emit('new-message', savedMessage);
    });

    // Handle typing indicators
    socket.on('typing', (data: { roomId: string; userId: string; isTyping: boolean }) => {
      socket.to(data.roomId).emit('user-typing', {
        userId: data.userId,
        isTyping: data.isTyping,
      });
    });

    socket.on('disconnect', () => {
      console.log('Client disconnected:', socket.id);
    });
  });

  (global as any).io = io;

  return NextResponse.json({ success: true, status: 'started' });
}
```

```typescript
// hooks/use-socket.ts
'use client';

import { useEffect, useRef, useCallback, useState } from 'react';
import { io, Socket } from 'socket.io-client';

type SocketOptions = {
  autoConnect?: boolean;
  reconnection?: boolean;
  reconnectionAttempts?: number;
};

export function useSocket(options: SocketOptions = {}) {
  const socketRef = useRef<Socket | null>(null);
  const [isConnected, setIsConnected] = useState(false);
  const [transport, setTransport] = useState('N/A');

  useEffect(() => {
    // Initialize socket connection
    const socket = io({
      path: '/api/socket/io',
      addTrailingSlash: false,
      autoConnect: options.autoConnect !== false,
      reconnection: options.reconnection !== false,
      reconnectionAttempts: options.reconnectionAttempts || 5,
    });

    socketRef.current = socket;

    function onConnect() {
      setIsConnected(true);
      setTransport(socket.io.engine?.transport?.name || 'N/A');
      
      // Upgrade to websocket if polling
      socket.io.engine?.on('upgrade', (transport) => {
        setTransport(transport.name);
      });
    }

    function onDisconnect() {
      setIsConnected(false);
      setTransport('N/A');
    }

    function onError(error: Error) {
      console.error('Socket error:', error);
    }

    socket.on('connect', onConnect);
    socket.on('disconnect', onDisconnect);
    socket.on('error', onError);

    // Cleanup
    return () => {
      socket.off('connect', onConnect);
      socket.off('disconnect', onDisconnect);
      socket.off('error', onError);
      socket.disconnect();
    };
  }, [options.autoConnect, options.reconnection, options.reconnectionAttempts]);

  const emit = useCallback((event: string, data: any) => {
    socketRef.current?.emit(event, data);
  }, []);

  const on = useCallback((event: string, callback: (data: any) => void) => {
    socketRef.current?.on(event, callback);
    return () => socketRef.current?.off(event, callback);
  }, []);

  const joinRoom = useCallback((roomId: string) => {
    socketRef.current?.emit('join-room', roomId);
  }, []);

  return {
    socket: socketRef.current,
    isConnected,
    transport,
    emit,
    on,
    joinRoom,
  };
}
```

### WebSocket Chat Component

Build a real-time chat interface:

```typescript
// components/chat/chat-room.tsx
'use client';

import { useEffect, useState, useRef } from 'react';
import { useSocket } from '@/hooks/use-socket';
import { useUser } from '@/hooks/use-user';
import { formatDistanceToNow } from 'date-fns';

interface Message {
  id: string;
  content: string;
  userId: string;
  user: { name: string; image: string };
  createdAt: string;
}

export function ChatRoom({ roomId }: { roomId: string }) {
  const { isConnected, on, emit, joinRoom } = useSocket();
  const { user } = useUser();
  const [messages, setMessages] = useState<Message[]>([]);
  const [inputValue, setInputValue] = useState('');
  const [typingUsers, setTypingUsers] = useState<Set<string>>(new Set());
  const messagesEndRef = useRef<HTMLDivElement>(null);
  const typingTimeoutRef = useRef<NodeJS.Timeout>();

  // Join room on mount
  useEffect(() => {
    if (isConnected) {
      joinRoom(roomId);
      
      // Load existing messages
      fetch(`/api/rooms/${roomId}/messages`)
        .then(r => r.json())
        .then(data => setMessages(data.messages));
    }
  }, [isConnected, roomId, joinRoom]);

  // Listen for new messages
  useEffect(() => {
    const unsubscribe = on('new-message', (message: Message) => {
      setMessages(prev => [...prev, message]);
    });

    return unsubscribe;
  }, [on]);

  // Listen for typing indicators
  useEffect(() => {
    const unsubscribe = on('user-typing', ({ userId, isTyping }: { userId: string; isTyping: boolean }) => {
      setTypingUsers(prev => {
        const next = new Set(prev);
        if (isTyping) {
          next.add(userId);
        } else {
          next.delete(userId);
        }
        return next;
      });
    });

    return unsubscribe;
  }, [on]);

  // Auto-scroll to bottom
  useEffect(() => {
    messagesEndRef.current?.scrollIntoView({ behavior: 'smooth' });
  }, [messages]);

  const handleSend = () => {
    if (!inputValue.trim() || !user) return;

    emit('send-message', {
      roomId,
      message: inputValue,
      userId: user.id,
    });

    setInputValue('');
    handleTyping(false);
  };

  const handleTyping = (isTyping: boolean) => {
    emit('typing', { roomId, userId: user?.id, isTyping });
    
    if (isTyping) {
      // Clear existing timeout
      if (typingTimeoutRef.current) {
        clearTimeout(typingTimeoutRef.current);
      }
      // Stop typing indicator after 3 seconds of inactivity
      typingTimeoutRef.current = setTimeout(() => {
        handleTyping(false);
      }, 3000);
    }
  };

  return (
    <div className="flex flex-col h-[600px] border rounded-lg">
      {/* Header */}
      <div className="p-4 border-b bg-gray-50">
        <div className="flex items-center gap-2">
          <div className={`w-2 h-2 rounded-full ${isConnected ? 'bg-green-500' : 'bg-red-500'}`} />
          <span className="text-sm text-gray-600">
            {isConnected ? 'Connected' : 'Disconnected'}
          </span>
        </div>
      </div>

      {/* Messages */}
      <div className="flex-1 overflow-y-auto p-4 space-y-4">
        {messages.map((message) => (
          <div
            key={message.id}
            className={`flex gap-3 ${
              message.userId === user?.id ? 'flex-row-reverse' : ''
            }`}
          >
            <img
              src={message.user.image}
              alt={message.user.name}
              className="w-8 h-8 rounded-full"
            />
            <div className={`max-w-[70%] ${
              message.userId === user?.id ? 'bg-blue-500 text-white' : 'bg-gray-100'
            } rounded-lg p-3`}>
              <p className="text-xs opacity-75 mb-1">{message.user.name}</p>
              <p>{message.content}</p>
              <p className="text-xs opacity-50 mt-1">
                {formatDistanceToNow(new Date(message.createdAt), { addSuffix: true })}
              </p>
            </div>
          </div>
        ))}
        <div ref={messagesEndRef} />
      </div>

      {/* Typing indicator */}
      {typingUsers.size > 0 && (
        <div className="px-4 py-2 text-sm text-gray-500 italic">
          Someone is typing...
        </div>
      )}

      {/* Input */}
      <div className="p-4 border-t">
        <div className="flex gap-2">
          <input
            type="text"
            value={inputValue}
            onChange={(e) => {
              setInputValue(e.target.value);
              handleTyping(true);
            }}
            onKeyDown={(e) => {
              if (e.key === 'Enter') handleSend();
            }}
            placeholder="Type a message..."
            className="flex-1 px-4 py-2 border rounded-lg"
          />
          <button
            onClick={handleSend}
            disabled={!isConnected || !inputValue.trim()}
            className="px-6 py-2 bg-blue-500 text-white rounded-lg disabled:opacity-50"
          >
            Send
          </button>
        </div>
      </div>
    </div>
  );
}
```

## 27.2 Server-Sent Events (SSE)

Server-Sent Events provide a simpler, HTTP-based alternative for unidirectional server-to-client streaming.

### SSE Route Handler

Implement SSE for live updates:

```typescript
// app/api/stream/updates/route.ts
export const dynamic = 'force-dynamic';

export async function GET(request: Request) {
  const encoder = new TextEncoder();
  let isClosed = false;

  // Create stream
  const stream = new ReadableStream({
    async start(controller) {
      // Send initial data
      const sendEvent = (data: any, event?: string) => {
        if (isClosed) return;
        
        let message = '';
        if (event) {
          message += `event: ${event}\n`;
        }
        message += `data: ${JSON.stringify(data)}\n\n`;
        
        controller.enqueue(encoder.encode(message));
      };

      // Send connection established event
      sendEvent({ connected: true, timestamp: Date.now() }, 'connected');

      // Simulate live data (replace with real data source)
      const interval = setInterval(async () => {
        try {
          // Fetch latest data
          const stats = await getRealtimeStats();
          sendEvent(stats, 'stats-update');
        } catch (error) {
          sendEvent({ error: 'Failed to fetch stats' }, 'error');
        }
      }, 5000);

      // Listen for client disconnect
      request.signal.addEventListener('abort', () => {
        isClosed = true;
        clearInterval(interval);
        controller.close();
      });
    },
  });

  return new Response(stream, {
    headers: {
      'Content-Type': 'text/event-stream',
      'Cache-Control': 'no-cache',
      'Connection': 'keep-alive',
      'X-Accel-Buffering': 'no', // Disable nginx buffering
    },
  });
}

async function getRealtimeStats() {
  // Replace with actual database queries or external API calls
  return {
    activeUsers: Math.floor(Math.random() * 1000),
    totalSales: Math.floor(Math.random() * 50000),
    timestamp: new Date().toISOString(),
  };
}
```

### SSE Hook

Consume Server-Sent Events in React:

```typescript
// hooks/use-server-sent-events.ts
'use client';

import { useEffect, useState, useCallback, useRef } from 'react';

interface SSEOptions {
  onMessage?: (data: any) => void;
  onError?: (error: Event) => void;
  onOpen?: () => void;
  events?: Record<string, (data: any) => void>;
}

export function useServerSentEvents(url: string | null, options: SSEOptions = {}) {
  const [isConnected, setIsConnected] = useState(false);
  const [lastEventId, setLastEventId] = useState<string | null>(null);
  const eventSourceRef = useRef<EventSource | null>(null);

  const connect = useCallback(() => {
    if (!url) return;

    // Close existing connection
    if (eventSourceRef.current) {
      eventSourceRef.current.close();
    }

    const es = new EventSource(url);
    eventSourceRef.current = es;

    es.onopen = () => {
      setIsConnected(true);
      options.onOpen?.();
    };

    es.onerror = (error) => {
      setIsConnected(false);
      options.onError?.(error);
      
      // Auto-reconnect after 5 seconds
      setTimeout(() => {
        if (eventSourceRef.current?.readyState === EventSource.CLOSED) {
          connect();
        }
      }, 5000);
    };

    es.onmessage = (event) => {
      try {
        const data = JSON.parse(event.data);
        setLastEventId(event.lastEventId);
        options.onMessage?.(data);
      } catch (e) {
        console.error('Failed to parse SSE message:', e);
      }
    };

    // Register specific event handlers
    if (options.events) {
      Object.entries(options.events).forEach(([eventName, handler]) => {
        es.addEventListener(eventName, (event) => {
          try {
            const data = JSON.parse(event.data);
            handler(data);
          } catch (e) {
            console.error(`Failed to parse ${eventName} event:`, e);
          }
        });
      });
    }

    return () => {
      es.close();
    };
  }, [url]);

  useEffect(() => {
    const cleanup = connect();
    return () => {
      cleanup?.();
      eventSourceRef.current?.close();
    };
  }, [connect]);

  const close = useCallback(() => {
    eventSourceRef.current?.close();
    setIsConnected(false);
  }, []);

  return {
    isConnected,
    lastEventId,
    close,
  };
}
```

### Live Dashboard with SSE

Build a real-time analytics dashboard:

```typescript
// components/live-dashboard.tsx
'use client';

import { useState } from 'react';
import { useServerSentEvents } from '@/hooks/use-server-sent-events';

interface Stats {
  activeUsers: number;
  totalSales: number;
  timestamp: string;
}

export function LiveDashboard() {
  const [stats, setStats] = useState<Stats | null>(null);
  const [events, setEvents] = useState<Array<{ type: string; data: any }>>([]);

  const { isConnected } = useServerSentEvents('/api/stream/updates', {
    onOpen: () => console.log('SSE Connected'),
    onError: (error) => console.error('SSE Error:', error),
    events: {
      'stats-update': (data) => {
        setStats(data);
        setEvents(prev => [...prev.slice(-9), { type: 'update', data }]);
      },
      'connected': (data) => {
        setEvents(prev => [...prev, { type: 'connected', data }]);
      },
      'error': (data) => {
        setEvents(prev => [...prev, { type: 'error', data }]);
      },
    },
  });

  return (
    <div className="p-6 space-y-6">
      <div className="flex items-center justify-between">
        <h1 className="text-2xl font-bold">Live Dashboard</h1>
        <div className="flex items-center gap-2">
          <div className={`w-3 h-3 rounded-full ${isConnected ? 'bg-green-500 animate-pulse' : 'bg-red-500'}`} />
          <span className="text-sm text-gray-600">
            {isConnected ? 'Live' : 'Disconnected'}
          </span>
        </div>
      </div>

      {stats && (
        <div className="grid grid-cols-2 gap-4">
          <div className="p-4 bg-blue-50 rounded-lg">
            <p className="text-sm text-blue-600 font-medium">Active Users</p>
            <p className="text-3xl font-bold text-blue-900">
              {stats.activeUsers.toLocaleString()}
            </p>
          </div>
          <div className="p-4 bg-green-50 rounded-lg">
            <p className="text-sm text-green-600 font-medium">Total Sales</p>
            <p className="text-3xl font-bold text-green-900">
              ${stats.totalSales.toLocaleString()}
            </p>
          </div>
        </div>
      )}

      <div className="border rounded-lg p-4 bg-gray-50">
        <h3 className="font-medium mb-2">Event Log</h3>
        <div className="space-y-2 max-h-48 overflow-y-auto">
          {events.map((event, i) => (
            <div key={i} className="text-xs font-mono p-2 bg-white rounded border">
              <span className={`font-bold ${
                event.type === 'error' ? 'text-red-500' : 
                event.type === 'connected' ? 'text-green-500' : 'text-blue-500'
              }`}>
                [{event.type}]
              </span>
              {' '}
              {JSON.stringify(event.data)}
            </div>
          ))}
        </div>
      </div>
    </div>
  );
}
```

## 27.3 Real-Time Data Sync

Synchronize data across clients using real-time databases and patterns.

### Real-Time Database with Supabase

Leverage Supabase Realtime for Postgres changes:

```typescript
// lib/supabase/realtime.ts
'use client';

import { useEffect } from 'react';
import { supabase } from './client';
import { RealtimeChannel } from '@supabase/supabase-js';

type UseRealtimeOptions = {
  table: string;
  event?: 'INSERT' | 'UPDATE' | 'DELETE' | '*';
  filter?: string;
  onData: (payload: any) => void;
};

export function useRealtime({ table, event = '*', filter, onData }: UseRealtimeOptions) {
  useEffect(() => {
    const channel: RealtimeChannel = supabase
      .channel(`public:${table}`)
      .on(
        'postgres_changes',
        {
          event,
          schema: 'public',
          table,
          filter,
        },
        (payload) => {
          onData(payload);
        }
      )
      .subscribe();

    return () => {
      supabase.removeChannel(channel);
    };
  }, [table, event, filter, onData]);
}

// Usage in component
// components/live-posts.tsx
'use client';

import { useState, useCallback } from 'react';
import { useRealtime } from '@/lib/supabase/realtime';

interface Post {
  id: string;
  title: string;
  content: string;
  created_at: string;
}

export function LivePosts() {
  const [posts, setPosts] = useState<Post[]>([]);

  const handleNewPost = useCallback((payload: any) => {
    if (payload.eventType === 'INSERT') {
      setPosts(prev => [payload.new, ...prev]);
    } else if (payload.eventType === 'UPDATE') {
      setPosts(prev => 
        prev.map(p => p.id === payload.new.id ? payload.new : p)
      );
    } else if (payload.eventType === 'DELETE') {
      setPosts(prev => prev.filter(p => p.id !== payload.old.id));
    }
  }, []);

  useRealtime({
    table: 'posts',
    event: '*',
    onData: handleNewPost,
  });

  return (
    <div className="space-y-4">
      {posts.map(post => (
        <div key={post.id} className="p-4 border rounded-lg">
          <h3 className="font-bold">{post.title}</h3>
          <p className="text-gray-600">{post.content}</p>
        </div>
      ))}
    </div>
  );
}
```

### Optimistic Updates with Rollback

Maintain UI responsiveness with optimistic patterns:

```typescript
// hooks/use-optimistic-realtime.ts
'use client';

import { useState, useCallback } from 'react';
import { useSWRConfig } from 'swr';

export function useOptimisticRealtime<T>(
  key: string,
  updater: (current: T[], item: T) => T[]
) {
  const { mutate } = useSWRConfig();
  const [pending, setPending] = useState<Map<string, T>>(new Map());

  const optimisticUpdate = useCallback(async (
    item: T,
    operation: () => Promise<void>,
    rollback: () => void
  ) => {
    const id = crypto.randomUUID();
    
    // Add to pending
    setPending(prev => new Map(prev.set(id, item)));
    
    // Optimistically update cache
    mutate(
      key,
      (current: T[]) => updater(current || [], item),
      false
    );

    try {
      await operation();
      setPending(prev => {
        const next = new Map(prev);
        next.delete(id);
        return next;
      });
    } catch (error) {
      // Rollback on error
      rollback();
      setPending(prev => {
        const next = new Map(prev);
        next.delete(id);
        return next;
      });
      throw error;
    }
  }, [key, mutate, updater]);

  return { optimisticUpdate, pending };
}

// Usage
const { optimisticUpdate } = useOptimisticRealtime<Post>(
  '/api/posts',
  (posts, newPost) => [newPost, ...posts]
);

const handleSubmit = async (content: string) => {
  const tempPost = { id: tempId, content, status: 'pending' };
  
  await optimisticUpdate(
    tempPost,
    () => fetch('/api/posts', { 
      method: 'POST', 
      body: JSON.stringify({ content }) 
    }).then(r => r.json()),
    () => {
      // Remove temp post on failure
      mutate('/api/posts', (posts) => 
        posts.filter((p: Post) => p.id !== tempId)
      );
    }
  );
};
```

## 27.4 Push Notifications

Implement browser push notifications for engagement.

### Service Worker Setup

Configure Next.js with service worker for push:

```typescript
// public/service-worker.js
self.addEventListener('push', (event) => {
  const data = event.data?.json() ?? {};
  
  const options = {
    body: data.body || 'New notification',
    icon: '/icon-192x192.png',
    badge: '/badge-72x72.png',
    tag: data.tag || 'default',
    requireInteraction: false,
    data: data.url || '/',
    actions: data.actions || [],
  };

  event.waitUntil(
    self.registration.showNotification(data.title || 'Update', options)
  );
});

self.addEventListener('notificationclick', (event) => {
  event.notification.close();
  
  event.waitUntil(
    clients.openWindow(event.notification.data)
  );
});
```

```typescript
// hooks/use-push-notifications.ts
'use client';

import { useState, useEffect, useCallback } from 'react';

export function usePushNotifications() {
  const [permission, setPermission] = useState<NotificationPermission>('default');
  const [subscription, setSubscription] = useState<PushSubscription | null>(null);

  useEffect(() => {
    if ('Notification' in window) {
      setPermission(Notification.permission);
    }
  }, []);

  const subscribe = useCallback(async () => {
    if (!('serviceWorker' in navigator) || !('PushManager' in window)) {
      throw new Error('Push notifications not supported');
    }

    // Request permission
    const result = await Notification.requestPermission();
    setPermission(result);
    
    if (result !== 'granted') {
      throw new Error('Permission denied');
    }

    // Register service worker
    const registration = await navigator.serviceWorker.register('/service-worker.js');
    
    // Subscribe to push
    const sub = await registration.pushManager.subscribe({
      userVisibleOnly: true,
      applicationServerKey: urlBase64ToUint8Array(
        process.env.NEXT_PUBLIC_VAPID_PUBLIC_KEY!
      ),
    });

    setSubscription(sub);

    // Send subscription to server
    await fetch('/api/notifications/subscribe', {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify(sub),
    });

    return sub;
  }, []);

  const unsubscribe = useCallback(async () => {
    if (!subscription) return;
    
    await subscription.unsubscribe();
    setSubscription(null);
    
    await fetch('/api/notifications/unsubscribe', {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify({ endpoint: subscription.endpoint }),
    });
  }, [subscription]);

  return {
    permission,
    subscription,
    subscribe,
    unsubscribe,
    isSupported: 'PushManager' in window,
  };
}

// Utility to convert VAPID key
function urlBase64ToUint8Array(base64String: string) {
  const padding = '='.repeat((4 - base64String.length % 4) % 4);
  const base64 = (base64String + padding)
    .replace(/\-/g, '+')
    .replace(/_/g, '/');
  
  const rawData = window.atob(base64);
  const outputArray = new Uint8Array(rawData.length);
  
  for (let i = 0; i < rawData.length; ++i) {
    outputArray[i] = rawData.charCodeAt(i);
  }
  return outputArray;
}
```

### Sending Push Notifications

Server-side push delivery:

```typescript
// app/api/notifications/send/route.ts
import webpush from 'web-push';

webpush.setVapidDetails(
  'mailto:admin@example.com',
  process.env.VAPID_PUBLIC_KEY!,
  process.env.VAPID_PRIVATE_KEY!
);

export async function POST(request: Request) {
  const { userId, title, body, url } = await request.json();

  // Get user subscriptions from database
  const subscriptions = await db.pushSubscription.findMany({
    where: { userId },
  });

  const notificationPayload = JSON.stringify({
    title,
    body,
    url,
    icon: '/icon-192x192.png',
  });

  // Send to all user devices
  const results = await Promise.allSettled(
    subscriptions.map((sub) =>
      webpush.sendNotification(
        {
          endpoint: sub.endpoint,
          keys: {
            p256dh: sub.p256dh,
            auth: sub.auth,
          },
        },
        notificationPayload
      )
    )
  );

  // Clean up invalid subscriptions
  const invalidSubs = subscriptions.filter((_, i) => 
    results[i].status === 'rejected' && 
    (results[i] as PromiseRejectedReason).reason.statusCode === 410
  );

  if (invalidSubs.length > 0) {
    await db.pushSubscription.deleteMany({
      where: {
        endpoint: { in: invalidSubs.map(s => s.endpoint) },
      },
    });
  }

  return Response.json({ 
    sent: results.filter(r => r.status === 'fulfilled').length,
    failed: results.filter(r => r.status === 'rejected').length,
  });
}
```

## 27.5 Collaborative Features

Build multi-user collaborative features like live cursors and shared editing.

### Live Cursors

Track and display other users' cursor positions:

```typescript
// components/collaborative/live-cursors.tsx
'use client';

import { useEffect, useState } from 'react';
import { useSocket } from '@/hooks/use-socket';
import { useThrottle } from '@/hooks/use-throttle';

interface Cursor {
  userId: string;
  name: string;
  color: string;
  x: number;
  y: number;
}

export function LiveCursors({ roomId }: { roomId: string }) {
  const { on, emit, isConnected } = useSocket();
  const [cursors, setCursors] = useState<Map<string, Cursor>>(new Map());
  const [myPosition, setMyPosition] = useState({ x: 0, y: 0 });

  // Throttle cursor updates to 20fps
  const throttledPosition = useThrottle(myPosition, 50);

  // Send my cursor position
  useEffect(() => {
    if (isConnected && throttledPosition) {
      emit('cursor-move', {
        roomId,
        x: throttledPosition.x,
        y: throttledPosition.y,
      });
    }
  }, [throttledPosition, isConnected, emit, roomId]);

  // Listen for other cursors
  useEffect(() => {
    const unsubscribe = on('cursor-update', (data: Cursor & { roomId: string }) => {
      if (data.roomId !== roomId) return;
      
      setCursors(prev => new Map(prev.set(data.userId, data)));
      
      // Remove cursor after inactivity
      setTimeout(() => {
        setCursors(prev => {
          const next = new Map(prev);
          next.delete(data.userId);
          return next;
        });
      }, 5000);
    });

    return unsubscribe;
  }, [on, roomId]);

  // Track mouse position
  useEffect(() => {
    const handleMouseMove = (e: MouseEvent) => {
      setMyPosition({ x: e.clientX, y: e.clientY });
    };

    window.addEventListener('mousemove', handleMouseMove);
    return () => window.removeEventListener('mousemove', handleMouseMove);
  }, []);

  return (
    <div className="fixed inset-0 pointer-events-none z-50">
      {Array.from(cursors.values()).map((cursor) => (
        <div
          key={cursor.userId}
          className="absolute transition-all duration-100 ease-out"
          style={{
            left: cursor.x,
            top: cursor.y,
            transform: 'translate(-50%, -50%)',
          }}
        >
          <svg
            width="24"
            height="24"
            viewBox="0 0 24 24"
            fill="none"
            style={{ color: cursor.color }}
          >
            <path
              d="M5.65376 12.3673H5.46026L5.31717 12.4976L0.500002 16.8829L0.500002 1.19841L11.7841 12.3673H5.65376Z"
              fill={cursor.color}
              stroke="white"
              strokeWidth="1"
            />
          </svg>
          <span
            className="absolute left-4 top-4 px-2 py-1 rounded text-xs text-white whitespace-nowrap"
            style={{ backgroundColor: cursor.color }}
          >
            {cursor.name}
          </span>
        </div>
      ))}
    </div>
  );
}
```

### Presence Awareness

Show who's currently viewing a document:

```typescript
// hooks/use-presence.ts
'use client';

import { useEffect, useState, useCallback } from 'react';
import { useSocket } from './use-socket';

interface User {
  id: string;
  name: string;
  image: string;
  color: string;
}

export function usePresence(roomId: string) {
  const { on, emit, isConnected } = useSocket();
  const [users, setUsers] = useState<User[]>([]);
  const [isTyping, setIsTyping] = useState<Set<string>>(new Set());

  useEffect(() => {
    if (!isConnected) return;

    // Join room and announce presence
    emit('presence:join', { roomId });

    const unsubscribeJoin = on('presence:joined', (user: User) => {
      setUsers(prev => [...prev.filter(u => u.id !== user.id), user]);
    });

    const unsubscribeLeave = on('presence:left', (userId: string) => {
      setUsers(prev => prev.filter(u => u.id !== userId));
    });

    const unsubscribeTyping = on('presence:typing', ({ userId, isTyping }: { userId: string; isTyping: boolean }) => {
      setIsTyping(prev => {
        const next = new Set(prev);
        if (isTyping) next.add(userId);
        else next.delete(userId);
        return next;
      });
    });

    return () => {
      emit('presence:leave', { roomId });
      unsubscribeJoin();
      unsubscribeLeave();
      unsubscribeTyping();
    };
  }, [roomId, isConnected, emit, on]);

  const setTyping = useCallback((typing: boolean) => {
    emit('presence:typing', { roomId, isTyping: typing });
  }, [emit, roomId]);

  return { users, isTyping, setTyping };
}
```

## 27.6 Real-Time Best Practices

Architecture patterns for scalable, reliable real-time systems.

### Connection Management

Handle connection states gracefully:

```typescript
// hooks/use-connection-status.ts
'use client';

import { useState, useEffect } from 'react';

export function useConnectionStatus() {
  const [status, setStatus] = useState<'online' | 'offline'>('online');
  const [wasOffline, setWasOffline] = useState(false);

  useEffect(() => {
    const handleOnline = () => {
      setStatus('online');
      if (wasOffline) {
        // Sync any queued data
        window.dispatchEvent(new CustomEvent('sync:pending'));
      }
      setWasOffline(false);
    };

    const handleOffline = () => {
      setStatus('offline');
      setWasOffline(true);
    };

    window.addEventListener('online', handleOnline);
    window.addEventListener('offline', handleOffline);

    // Initial check
    setStatus(navigator.onLine ? 'online' : 'offline');

    return () => {
      window.removeEventListener('online', handleOnline);
      window.removeEventListener('offline', handleOffline);
    };
  }, [wasOffline]);

  return { status, wasOffline };
}

// Queue actions when offline
// hooks/use-offline-queue.ts
'use client';

import { useCallback, useEffect, useRef } from 'react';

type QueueItem = {
  id: string;
  action: () => Promise<void>;
  retryCount: number;
};

export function useOfflineQueue() {
  const queueRef = useRef<QueueItem[]>([]);

  const enqueue = useCallback((action: () => Promise<void>) => {
    const item: QueueItem = {
      id: crypto.randomUUID(),
      action,
      retryCount: 0,
    };

    if (!navigator.onLine) {
      queueRef.current.push(item);
      // Persist to localStorage for page refreshes
      persistQueue();
    } else {
      // Execute immediately if online
      executeItem(item);
    }
  }, []);

  const executeItem = async (item: QueueItem) => {
    try {
      await item.action();
    } catch (error) {
      if (item.retryCount < 3) {
        item.retryCount++;
        queueRef.current.push(item);
        setTimeout(() => executeItem(item), 1000 * item.retryCount);
      }
    }
  };

  const processQueue = useCallback(() => {
    const items = [...queueRef.current];
    queueRef.current = [];
    localStorage.removeItem('offline-queue');
    
    items.forEach(item => executeItem(item));
  }, []);

  const persistQueue = () => {
    localStorage.setItem('offline-queue', JSON.stringify(queueRef.current));
  };

  useEffect(() => {
    // Restore queue on mount
    const saved = localStorage.getItem('offline-queue');
    if (saved) {
      queueRef.current = JSON.parse(saved);
    }

    // Process when coming back online
    const handleSync = () => processQueue();
    window.addEventListener('sync:pending', handleSync);

    return () => window.removeEventListener('sync:pending', handleSync);
  }, [processQueue]);

  return { enqueue, processQueue };
}
```

### Rate Limiting and Backpressure

Prevent server overload:

```typescript
// lib/rate-limiter.ts
import { LRUCache } from 'lru-cache';

interface RateLimitConfig {
  windowMs: number;
  maxRequests: number;
}

export function createRateLimiter(config: RateLimitConfig) {
  const cache = new LRUCache<string, number[]>({
    max: 500,
    ttl: config.windowMs,
  });

  return {
    check: (key: string): { allowed: boolean; remaining: number; resetTime: number } => {
      const now = Date.now();
      const timestamps = cache.get(key) || [];
      
      // Remove old timestamps
      const validTimestamps = timestamps.filter(t => now - t < config.windowMs);
      
      if (validTimestamps.length >= config.maxRequests) {
        const oldestTimestamp = validTimestamps[0];
        return {
          allowed: false,
          remaining: 0,
          resetTime: oldestTimestamp + config.windowMs,
        };
      }

      validTimestamps.push(now);
      cache.set(key, validTimestamps);

      return {
        allowed: true,
        remaining: config.maxRequests - validTimestamps.length,
        resetTime: now + config.windowMs,
      };
    },
  };
}

// Usage in Socket.io
const messageLimiter = createRateLimiter({ windowMs: 60000, maxRequests: 30 });

io.on('connection', (socket) => {
  socket.on('send-message', (data) => {
    const result = messageLimiter.check(socket.id);
    
    if (!result.allowed) {
      socket.emit('error', { 
        message: 'Rate limit exceeded', 
        retryAfter: Math.ceil((result.resetTime - Date.now()) / 1000)
      });
      return;
    }
    
    // Process message...
  });
});
```

### Security for Real-Time

Authenticate WebSocket connections:

```typescript
// middleware/socket-auth.ts
import { Socket } from 'socket.io';
import { verifyToken } from '@/lib/auth';

export async function authenticateSocket(socket: Socket, next: (err?: Error) => void) {
  try {
    const token = socket.handshake.auth.token || socket.handshake.headers.authorization?.split(' ')[1];
    
    if (!token) {
      return next(new Error('Authentication required'));
    }

    const payload = await verifyToken(token);
    if (!payload) {
      return next(new Error('Invalid token'));
    }

    // Attach user data to socket
    socket.data.user = payload;
    socket.data.userId = payload.sub;
    
    next();
  } catch (error) {
    next(new Error('Authentication failed'));
  }
}

// Usage
io.use(authenticateSocket);

io.on('connection', (socket) => {
  // Now socket.data.user is available
  console.log('User connected:', socket.data.userId);
  
  // Only allow joining rooms user has access to
  socket.on('join-room', async (roomId) => {
    const hasAccess = await checkRoomAccess(socket.data.userId, roomId);
    if (!hasAccess) {
      socket.emit('error', { message: 'Access denied' });
      return;
    }
    
    socket.join(roomId);
  });
});
```

## Key Takeaways from Chapter 27

1. **WebSockets with Socket.io**: Initialize Socket.io in a Route Handler and store the instance on `global` to prevent multiple instances during hot reloading. Use rooms to namespace connections and broadcast messages to specific groups. Implement reconnection logic on the client with exponential backoff.

2. **Server-Sent Events (SSE)**: Use SSE for unidirectional server-to-client streaming (live dashboards, notifications) using standard HTTP. Implement the ReadableStream API in Route Handlers with proper headers (`text/event-stream`, `no-cache`). Handle client disconnects via `request.signal.addEventListener('abort')`.

3. **Real-Time Data Sync**: Leverage database-specific real-time features (Supabase Realtime, Firebase, Convex) to subscribe to data changes directly. Implement optimistic updates with rollback capabilities for responsive UIs, using SWR or React Query's optimistic mutation patterns.

4. **Push Notifications**: Register a Service Worker for background push handling, subscribe users via the Push API with VAPID keys, and send notifications using web-push libraries. Clean up invalid subscriptions (410 Gone) from your database to prevent retry loops.

5. **Collaborative Features**: Track user presence with heartbeat mechanisms, throttle cursor position updates (50ms), and use operational transforms or CRDTs for conflict resolution in collaborative editing. Display visual indicators (cursors, user avatars) to enhance the sense of shared space.

6. **Connection Management**: Monitor online/offline status with `navigator.onLine`, queue actions during disconnection using localStorage persistence, and process the queue when connectivity returns. Implement rate limiting on both client and server to prevent spam and server overload.

7. **Security**: Authenticate WebSocket connections during the handshake using JWT tokens, validate room access permissions before allowing joins, and never trust client-sent data without server validation. Use rate limiting per user ID rather than connection ID to prevent connection cycling attacks.

## Coming Up Next

**Chapter 28: Advanced Caching Strategies**

Now that your application handles real-time data effectively, it's time to optimize how that data is cached and served. In Chapter 28, we'll explore multi-layer caching architectures, CDN integration strategies, edge caching patterns, cache invalidation techniques, stale-while-revalidate implementations, distributed caching with Redis, and cache performance monitoring. You'll learn how to build systems that serve data lightning-fast while maintaining consistency across global deployments.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='26. advanced_component_patterns.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='28. advanced_caching_strategies.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
